Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [32]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model. Image batch - conv2 - relu - conv2 - relu - W1X+B1 - relu - W2X+B2
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    print(conv.get_shape())
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]]) # reshape for fully connected
    print(reshape.get_shape())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

(16, 14, 14, 16)
(16, 784)
(10000, 14, 14, 16)
(10000, 784)
(10000, 14, 14, 16)
(10000, 784)


In [36]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  A = test_prediction.eval()
  print(A[40,:])
  print(test_labels[40,:])

Initialized
Minibatch loss at step 0: 2.875975
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.554699
Minibatch accuracy: 50.0%
Validation accuracy: 48.0%
Minibatch loss at step 100: 1.265032
Minibatch accuracy: 50.0%
Validation accuracy: 73.1%
Minibatch loss at step 150: 1.112460
Minibatch accuracy: 62.5%
Validation accuracy: 74.8%
Minibatch loss at step 200: 0.705411
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 250: 0.731804
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 300: 0.763030
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.169376
Minibatch accuracy: 93.8%
Validation accuracy: 79.2%
Minibatch loss at step 400: 0.479746
Minibatch accuracy: 81.2%
Validation accuracy: 78.7%
Minibatch loss at step 450: 0.646857
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.402931
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model. Image batch - conv2 - relu - conv2 - relu - W1X+B1 - relu - W2X+B2
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    print(conv.get_shape())
    max_pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(max_pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print(conv.get_shape())
    max_pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = max_pool.get_shape().as_list()
    reshape = tf.reshape(max_pool, [shape[0], shape[1] * shape[2] * shape[3]]) # reshape for fully connected
    print(reshape.get_shape())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

(16, 28, 28, 16)
(16, 14, 14, 16)
(16, 784)
(10000, 28, 28, 16)
(10000, 14, 14, 16)
(10000, 784)
(10000, 28, 28, 16)
(10000, 14, 14, 16)
(10000, 784)


In [15]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 4.044528
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.468104
Minibatch accuracy: 43.8%
Validation accuracy: 64.4%
Minibatch loss at step 200: 1.135785
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%
Minibatch loss at step 300: 0.744451
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 400: 0.431003
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 500: 0.320825
Minibatch accuracy: 93.8%
Validation accuracy: 81.7%
Minibatch loss at step 600: 0.369904
Minibatch accuracy: 93.8%
Validation accuracy: 81.7%
Minibatch loss at step 700: 1.010833
Minibatch accuracy: 68.8%
Validation accuracy: 79.9%
Minibatch loss at step 800: 0.594602
Minibatch accuracy: 81.2%
Validation accuracy: 83.9%
Minibatch loss at step 900: 0.189898
Minibatch accuracy: 87.5%
Validation accuracy: 83.3%
Minibatch loss at step 1000: 0.514579
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [85]:
# TEST with DROPout, pool, REGULARIZATION, LEARNING RATE DECAY
import math as math
batch_size = 32
patch_size = 5
depth = 16
num_hidden = 64
beta = 1e-6
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], 
                                                   stddev=math.sqrt(2.0/(patch_size*patch_size*num_channels))))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth*2], 
                                                   stddev=math.sqrt(2.0/(patch_size*patch_size*depth))))
  layer2_biases = tf.Variable(tf.zeros(shape=[depth*2]))
  layer3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth*2, depth*4], 
                                                   stddev=math.sqrt(2.0/(patch_size*patch_size*depth*2))))
  layer3_biases = tf.Variable(tf.zeros(shape=[depth*4]))
  
  layer4_weights = tf.Variable(tf.truncated_normal([image_size // 7 * image_size // 7 * depth * 4 , num_hidden], 
                                                   stddev=math.sqrt(2.0/(image_size // 7 * image_size // 7 * depth * 4 ))))
  layer4_biases = tf.Variable(tf.zeros(shape=[num_hidden]))
  layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.zeros(shape=[num_labels]))
  
  def model(data, dropout = True):
    if dropout:
        data = tf.nn.dropout(data, 0.5)
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    max_pool = tf.nn.avg_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(max_pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    max_pool = tf.nn.avg_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(max_pool, layer3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer3_biases)
    max_pool = tf.nn.avg_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    shape = max_pool.get_shape().as_list()
    reshape = tf.reshape(max_pool, [shape[0], shape[1] * shape[2] * shape[3]]) # reshape for fully connected
    hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    #if dropout:
    #    hidden = tf.nn.dropout(hidden, 0.5)
    
    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularization = beta*(tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) + 
                         tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) +
                         tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) +
                         tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases) +
                         tf.nn.l2_loss(layer5_weights) + tf.nn.l2_loss(layer5_biases))
  loss = loss + regularization   
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 1000, 0.96, staircase=True)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [86]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 2.315977
Minibatch accuracy: 6.2%
Validation accuracy: 14.1%
Minibatch loss at step 1000: 0.357845
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 0.296639
Minibatch accuracy: 90.6%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 0.278882
Minibatch accuracy: 90.6%
Validation accuracy: 87.1%
Minibatch loss at step 4000: 0.340449
Minibatch accuracy: 90.6%
Validation accuracy: 87.3%
Minibatch loss at step 5000: 0.274941
Minibatch accuracy: 90.6%
Validation accuracy: 87.8%
Minibatch loss at step 6000: 0.073652
Minibatch accuracy: 96.9%
Validation accuracy: 87.8%
Minibatch loss at step 7000: 0.742682
Minibatch accuracy: 78.1%
Validation accuracy: 88.3%
Minibatch loss at step 8000: 0.404281
Minibatch accuracy: 90.6%
Validation accuracy: 88.7%
Minibatch loss at step 9000: 0.359289
Minibatch accuracy: 90.6%
Validation accuracy: 88.9%
Minibatch loss at step 10000: 0.538210
Minibatch accuracy: 84.4%
Validation accura